### Lecture

In [137]:
import pandas as pd
import numpy as np
import pyspark

In [2]:
# make the spark object 
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
spark

In [4]:
import pyspark.sql.functions as F

In [5]:
spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [6]:
np.random.seed(456)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list("abc"), 20))
)
pandas_dataframe

,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


In [7]:
df = spark.createDataFrame(pandas_dataframe)

In [8]:
# shows us the object not the actual df
df

DataFrame[n: bigint, group: string]

In [9]:
# have to call a method to actually see the df
df.show()

+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
|  5|    c|
|  6|    a|
|  7|    b|
|  8|    a|
|  9|    b|
| 10|    b|
| 11|    a|
| 12|    b|
| 13|    a|
| 14|    b|
| 15|    b|
| 16|    c|
| 17|    c|
| 18|    a|
| 19|    c|
+---+-----+



In [10]:
# we'll see this is a column object 
df.group

Column<b'group'>

In [11]:
type(df.group)

pyspark.sql.column.Column

In [12]:
# dataframe object, spark is still being lazy
df.select(df.group)

DataFrame[group: string]

In [13]:
df.select(df.group).show()

+-----+
|group|
+-----+
|    b|
|    b|
|    c|
|    a|
|    c|
|    c|
|    a|
|    b|
|    a|
|    b|
|    b|
|    a|
|    b|
|    a|
|    b|
|    b|
|    c|
|    c|
|    a|
|    c|
+-----+



In [14]:
df.select(df.n).show()

+---+
|  n|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



In [15]:
df.select(df.n + 1).show()

+-------+
|(n + 1)|
+-------+
|      1|
|      2|
|      3|
|      4|
|      5|
|      6|
|      7|
|      8|
|      9|
|     10|
|     11|
|     12|
|     13|
|     14|
|     15|
|     16|
|     17|
|     18|
|     19|
|     20|
+-------+



In [16]:
# this is a transformation
n_increment = df.n + 1
n_increment

Column<b'(n + 1)'>

In [17]:
# .show() is the action
df.select(n_increment).show()

+-------+
|(n + 1)|
+-------+
|      1|
|      2|
|      3|
|      4|
|      5|
|      6|
|      7|
|      8|
|      9|
|     10|
|     11|
|     12|
|     13|
|     14|
|     15|
|     16|
|     17|
|     18|
|     19|
|     20|
+-------+



In [18]:
# the fact that the transformation can exist independently of each other 
# spark df are immutable 

In [19]:
# transformation
df.describe()

DataFrame[summary: string, n: string, group: string]

In [20]:
df.describe().show()

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



In [21]:
df.describe().n

Column<b'n'>

In [22]:
df.describe().select('n', 'summary').show()

+-----------------+-------+
|                n|summary|
+-----------------+-------+
|               20|  count|
|              9.5|   mean|
|5.916079783099616| stddev|
|                0|    min|
|               19|    max|
+-----------------+-------+



In [162]:
from pydataset import data
data('mpg').head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [24]:
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [25]:
# can reference columns through sting or df.column name
mpg.select('model', 'manufacturer', 'hwy').show()

+------------------+------------+---+
|             model|manufacturer|hwy|
+------------------+------------+---+
|                a4|        audi| 29|
|                a4|        audi| 29|
|                a4|        audi| 31|
|                a4|        audi| 30|
|                a4|        audi| 26|
|                a4|        audi| 26|
|                a4|        audi| 27|
|        a4 quattro|        audi| 26|
|        a4 quattro|        audi| 25|
|        a4 quattro|        audi| 28|
|        a4 quattro|        audi| 27|
|        a4 quattro|        audi| 25|
|        a4 quattro|        audi| 25|
|        a4 quattro|        audi| 25|
|        a4 quattro|        audi| 25|
|        a6 quattro|        audi| 24|
|        a6 quattro|        audi| 25|
|        a6 quattro|        audi| 23|
|c1500 suburban 2wd|   chevrolet| 20|
|c1500 suburban 2wd|   chevrolet| 15|
+------------------+------------+---+
only showing top 20 rows



In [26]:
mpg.select('model', 'manufacturer', mpg.hwy.alias("highway_mileage")).show()

+------------------+------------+---------------+
|             model|manufacturer|highway_mileage|
+------------------+------------+---------------+
|                a4|        audi|             29|
|                a4|        audi|             29|
|                a4|        audi|             31|
|                a4|        audi|             30|
|                a4|        audi|             26|
|                a4|        audi|             26|
|                a4|        audi|             27|
|        a4 quattro|        audi|             26|
|        a4 quattro|        audi|             25|
|        a4 quattro|        audi|             28|
|        a4 quattro|        audi|             27|
|        a4 quattro|        audi|             25|
|        a4 quattro|        audi|             25|
|        a4 quattro|        audi|             25|
|        a4 quattro|        audi|             25|
|        a6 quattro|        audi|             24|
|        a6 quattro|        audi|             25|


In [27]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [28]:
mpg.select(((mpg.cty + mpg.hwy)/ 2).alias('avg_mileage')).show()

+-----------+
|avg_mileage|
+-----------+
|       23.5|
|       25.0|
|       25.5|
|       25.5|
|       21.0|
|       22.0|
|       22.5|
|       22.0|
|       20.5|
|       24.0|
|       23.0|
|       20.0|
|       21.0|
|       21.0|
|       20.0|
|       19.5|
|       21.0|
|       19.5|
|       17.0|
|       13.0|
+-----------+
only showing top 20 rows



In [29]:
# transformation
avg_mileage_col = ((mpg.cty + mpg.hwy)/ 2).alias('avg_mileage')
# .show() action
mpg.select('*', avg_mileage_col).show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+-----------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|avg_mileage|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+-----------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|       23.5|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|       25.0|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|       25.5|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|       25.5|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|       21.0|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|       22.0|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|       22.5|
|        audi|        a4 quattro|  1.8|1

In [30]:
# string --> string, double --> float, long --> int
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [31]:
from pyspark.sql.functions import col

In [32]:
col

<function pyspark.sql.functions._create_function.<locals>._(col)>

In [33]:
mpg.select(col('hwy'))

DataFrame[hwy: bigint]

In [34]:
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [35]:
from pyspark.sql.functions import expr

In [36]:
mpg.select(expr('hwy AS highway_average')).show(5)

+---------------+
|highway_average|
+---------------+
|             29|
|             29|
|             31|
|             30|
|             26|
+---------------+
only showing top 5 rows



In [37]:
# allows you to write sql queries instead of python
mpg.createOrReplaceTempView('mpg')

In [38]:
spark.sql("""
SELECT * FROM mpg
""").show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [39]:
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [40]:
mpg.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [41]:
mpg.select(mpg.manufacturer, mpg.model, mpg.cyl, mpg.cyl.cast('string').alias('cyl_string')).show()

+------------+------------------+---+----------+
|manufacturer|             model|cyl|cyl_string|
+------------+------------------+---+----------+
|        audi|                a4|  4|         4|
|        audi|                a4|  4|         4|
|        audi|                a4|  4|         4|
|        audi|                a4|  4|         4|
|        audi|                a4|  6|         6|
|        audi|                a4|  6|         6|
|        audi|                a4|  6|         6|
|        audi|        a4 quattro|  4|         4|
|        audi|        a4 quattro|  4|         4|
|        audi|        a4 quattro|  4|         4|
|        audi|        a4 quattro|  4|         4|
|        audi|        a4 quattro|  6|         6|
|        audi|        a4 quattro|  6|         6|
|        audi|        a4 quattro|  6|         6|
|        audi|        a4 quattro|  6|         6|
|        audi|        a6 quattro|  6|         6|
|        audi|        a6 quattro|  6|         6|
|        audi|      

In [42]:
# be careful with this 
mpg.select(mpg.manufacturer.cast('double')).show(5)

+------------+
|manufacturer|
+------------+
|        null|
|        null|
|        null|
|        null|
|        null|
+------------+
only showing top 5 rows



In [43]:
from pyspark.sql.functions import min, max

In [44]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [45]:
mpg.select(min(mpg.cyl).alias('min_cyl'), max(mpg.hwy)).show(5)

+-------+--------+
|min_cyl|max(hwy)|
+-------+--------+
|      4|      44|
+-------+--------+



In [46]:
spark.sql('SELECT min(cyl), max(hwy) FROM mpg').show(5)

+--------+--------+
|min(cyl)|max(hwy)|
+--------+--------+
|       4|      44|
+--------+--------+



In [138]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
        }
    )
)

textdf.show(truncate=False)

+---------------------------------------------+
|address                                      |
+---------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |
|303 Pearl Pkwy, San Antonio, TX 78215        |
|1255 SW Loop 410, San Antonio, TX 78227      |
+---------------------------------------------+



In [ ]:
# extract house/unit number
# extract street

In [143]:
from pyspark.sql.functions import regexp_extract, regexp_replace
# works similar to pandas series str.extract/replace

In [156]:
textdf.select(
    'address',
    regexp_extract('address', r'^(\d+)', 1).alias('unit_no'),
    regexp_extract('address', r'^\d+\s+(.*?),', 1).alias('street'),
    regexp_replace('address', r'^\d+\s+.*?,\s+', '').alias('city_state'),
    regexp_extract('address', r'(\d+(-\d+)?)$', 1).alias('zipcode')
                  ).show(truncate=False)
# 1 is one capture group

+---------------------------------------------+-------+------------------+---------------------+-------+
|address                                      |unit_no|street            |city_state           |zipcode|
+---------------------------------------------+-------+------------------+---------------------+-------+
|600 Navarro St ste 600, San Antonio, TX 78205|600    |Navarro St ste 600|San Antonio, TX 78205|78205  |
|3130 Broadway St, San Antonio, TX 78209      |3130   |Broadway St       |San Antonio, TX 78209|78209  |
|303 Pearl Pkwy, San Antonio, TX 78215        |303    |Pearl Pkwy        |San Antonio, TX 78215|78215  |
|1255 SW Loop 410, San Antonio, TX 78227      |1255   |SW Loop 410       |San Antonio, TX 78227|78227  |
+---------------------------------------------+-------+------------------+---------------------+-------+



In [158]:
# filtering data
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [163]:
mpg = spark.createDataFrame(data("mpg"))

In [166]:
four_cyl = mpg.cyl == 4
mpg.filter(four_cyl).show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [167]:
# there is no difference between filter and where 
from pyspark.sql.functions import when

In [172]:
mpg.select(
    'cty',
    when(mpg.cty >= 20, 'good gas mileage').otherwise('bad gas mileage').alias('good or bad mileage')

).show()

+---+-------------------+
|cty|good or bad mileage|
+---+-------------------+
| 18|    bad gas mileage|
| 21|   good gas mileage|
| 20|   good gas mileage|
| 21|   good gas mileage|
| 16|    bad gas mileage|
| 18|    bad gas mileage|
| 18|    bad gas mileage|
| 18|    bad gas mileage|
| 16|    bad gas mileage|
| 20|   good gas mileage|
| 19|    bad gas mileage|
| 15|    bad gas mileage|
| 17|    bad gas mileage|
| 17|    bad gas mileage|
| 15|    bad gas mileage|
| 15|    bad gas mileage|
| 17|    bad gas mileage|
| 16|    bad gas mileage|
| 14|    bad gas mileage|
| 11|    bad gas mileage|
+---+-------------------+
only showing top 20 rows



In [173]:
mpg.select(
    mpg.hwy,
    when(mpg.hwy > 25, "good_mileage")
    .otherwise("bad_mileage")
    .alias("mpg_desc"),
).show(12)

+---+------------+
|hwy|    mpg_desc|
+---+------------+
| 29|good_mileage|
| 29|good_mileage|
| 31|good_mileage|
| 30|good_mileage|
| 26|good_mileage|
| 26|good_mileage|
| 27|good_mileage|
| 26|good_mileage|
| 25| bad_mileage|
| 28|good_mileage|
| 27|good_mileage|
| 25| bad_mileage|
+---+------------+
only showing top 12 rows



In [174]:
mpg.select(
    mpg.displ,
    (
        when(mpg.displ < 2, "small")
        .when(mpg.displ < 3, "medium")
        .otherwise("large")
        .alias("engine_size")
    ),
).show(10)
# similar to python if, elif, else statement

+-----+-----------+
|displ|engine_size|
+-----+-----------+
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
|  2.0|     medium|
|  2.8|     medium|
|  2.8|     medium|
|  3.1|      large|
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
+-----+-----------+
only showing top 10 rows



In [234]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [235]:
mpg.sort('hwy').show()

+------------+--------------------+-----+----+---+----------+---+---+---+---+------+
|manufacturer|               model|displ|year|cyl|     trans|drv|cty|hwy| fl| class|
+------------+--------------------+-----+----+---+----------+---+---+---+---+------+
|        jeep|  grand cherokee 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge| ram 1500 pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|       dodge|   dakota pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|       dodge| ram 1500 pickup 4wd|  4.7|2008|  8|manual(m6)|  4|  9| 12|  e|pickup|
|       dodge|         durango 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|        jeep|  grand cherokee 4wd|  6.1|2008|  8|  auto(l5)|  4| 11| 14|  p|   suv|
|   chevrolet|     k1500 tahoe 4wd|  5.3|2008|  8|  auto(l4)|  4| 11| 14|  e|   suv|
|        ford|     f150 pickup 4wd|  5.4|1999|  8|  auto(l4)|  4| 11| 15|  r|pickup|
|       dodge|         durango 4wd|  5.9|1999|  8|  auto(l4)|  4|

In [236]:
mpg.orderBy('hwy').show()

+------------+--------------------+-----+----+---+----------+---+---+---+---+------+
|manufacturer|               model|displ|year|cyl|     trans|drv|cty|hwy| fl| class|
+------------+--------------------+-----+----+---+----------+---+---+---+---+------+
|       dodge|   dakota pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|        jeep|  grand cherokee 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|         durango 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge| ram 1500 pickup 4wd|  4.7|2008|  8|manual(m6)|  4|  9| 12|  e|pickup|
|       dodge| ram 1500 pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|        jeep|  grand cherokee 4wd|  6.1|2008|  8|  auto(l5)|  4| 11| 14|  p|   suv|
|   chevrolet|     k1500 tahoe 4wd|  5.3|2008|  8|  auto(l4)|  4| 11| 14|  e|   suv|
|  land rover|         range rover|  4.6|1999|  8|  auto(l4)|  4| 11| 15|  p|   suv|
|       dodge| ram 1500 pickup 4wd|  5.2|1999|  8|  auto(l4)|  4|

In [237]:
mpg.orderBy(asc('hwy')).show()

+------------+--------------------+-----+----+---+----------+---+---+---+---+------+
|manufacturer|               model|displ|year|cyl|     trans|drv|cty|hwy| fl| class|
+------------+--------------------+-----+----+---+----------+---+---+---+---+------+
|       dodge| ram 1500 pickup 4wd|  4.7|2008|  8|manual(m6)|  4|  9| 12|  e|pickup|
|       dodge|   dakota pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|        jeep|  grand cherokee 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge| ram 1500 pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|       dodge|         durango 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|        jeep|  grand cherokee 4wd|  6.1|2008|  8|  auto(l5)|  4| 11| 14|  p|   suv|
|   chevrolet|     k1500 tahoe 4wd|  5.3|2008|  8|  auto(l4)|  4| 11| 14|  e|   suv|
|       dodge|   dakota pickup 4wd|  5.2|1999|  8|  auto(l4)|  4| 11| 15|  r|pickup|
|       dodge| ram 1500 pickup 4wd|  5.2|1999|  8|  auto(l4)|  4|

In [238]:
mpg.sort(mpg.hwy.desc()).show()

+------------+------------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|       model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+------------+-----+----+---+----------+---+---+---+---+----------+
|  volkswagen|  new beetle|  1.9|1999|  4|manual(m5)|  f| 35| 44|  d|subcompact|
|  volkswagen|       jetta|  1.9|1999|  4|manual(m5)|  f| 33| 44|  d|   compact|
|  volkswagen|  new beetle|  1.9|1999|  4|  auto(l4)|  f| 29| 41|  d|subcompact|
|      toyota|     corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|       honda|       civic|  1.8|2008|  4|  auto(l5)|  f| 25| 36|  r|subcompact|
|       honda|       civic|  1.8|2008|  4|  auto(l5)|  f| 24| 36|  c|subcompact|
|      toyota|     corolla|  1.8|1999|  4|manual(m5)|  f| 26| 35|  r|   compact|
|      toyota|     corolla|  1.8|2008|  4|  auto(l4)|  f| 26| 35|  r|   compact|
|       honda|       civic|  1.8|2008|  4|manual(m5)|  f| 26| 34|  r|subcompact|
|      toyota|     corolla| 

In [ ]:
mpg.hwy.desc_nulls_last

In [239]:
mpg.groupBy('cyl').mean().show()

+---+------------------+------------------+--------+------------------+-----------------+
|cyl|        avg(displ)|         avg(year)|avg(cyl)|          avg(cty)|         avg(hwy)|
+---+------------------+------------------+--------+------------------+-----------------+
|  6| 3.408860759493671| 2002.873417721519|     6.0| 16.21518987341772|22.82278481012658|
|  5|               2.5|            2008.0|     5.0|              20.5|            28.75|
|  8|5.1328571428571435|2004.5285714285715|     8.0|12.571428571428571|17.62857142857143|
|  4| 2.145679012345679|            2003.0|     4.0|21.012345679012345|28.80246913580247|
+---+------------------+------------------+--------+------------------+-----------------+



In [240]:
mpg.groupBy('cyl', 'class').mean().show()

+---+----------+------------------+------------------+--------+------------------+------------------+
|cyl|     class|        avg(displ)|         avg(year)|avg(cyl)|          avg(cty)|          avg(hwy)|
+---+----------+------------------+------------------+--------+------------------+------------------+
|  5|   compact|               2.5|            2008.0|     5.0|              21.0|              29.0|
|  5|subcompact|               2.5|            2008.0|     5.0|              20.0|              28.5|
|  6|subcompact| 3.385714285714286|2005.4285714285713|     6.0|              17.0|24.714285714285715|
|  6|    pickup|3.8400000000000007|            2002.6|     6.0|              14.5|              17.9|
|  4|subcompact|1.9333333333333333|2001.5714285714287|     4.0|22.857142857142858| 30.80952380952381|
|  8|       suv| 5.155263157894736|2004.4473684210527|     8.0|12.131578947368421|16.789473684210527|
|  8|    pickup| 4.965000000000001|            2004.4|     8.0|              11.8|

In [241]:
mpg.groupBy('cyl', 'class').agg(mean('hwy'), mean('cty'), max('displ'), min('displ')).show()

+---+----------+------------------+------------------+----------+----------+
|cyl|     class|          avg(hwy)|          avg(cty)|max(displ)|min(displ)|
+---+----------+------------------+------------------+----------+----------+
|  5|   compact|              29.0|              21.0|       2.5|       2.5|
|  5|subcompact|              28.5|              20.0|       2.5|       2.5|
|  6|subcompact|24.714285714285715|              17.0|       4.0|       2.7|
|  6|    pickup|              17.9|              14.5|       4.2|       3.4|
|  4|subcompact| 30.80952380952381|22.857142857142858|       2.5|       1.6|
|  8|       suv|16.789473684210527|12.131578947368421|       6.5|       4.0|
|  8|    pickup|              15.8|              11.8|       5.9|       4.6|
|  8|   midsize|              24.0|              16.0|       5.3|       4.2|
|  4|   midsize|           29.1875|              20.5|       2.5|       1.8|
|  8|   2seater|              24.8|              15.4|       7.0|       5.7|

In [242]:
# groupBy the rows, .pivot the categorical var that will be columns, then agg
mpg.groupBy('class').pivot('cyl').count().show()

+----------+----+----+----+----+
|     class|   4|   5|   6|   8|
+----------+----+----+----+----+
|subcompact|  21|   2|   7|   5|
|   compact|  32|   2|  13|null|
|   minivan|   1|null|  10|null|
|       suv|   8|null|  16|  38|
|   midsize|  16|null|  23|   2|
|    pickup|   3|null|  10|  20|
|   2seater|null|null|null|   5|
+----------+----+----+----+----+



In [243]:
mpg.rollup('cyl').agg(mean('hwy')).show()

+----+-----------------+
| cyl|         avg(hwy)|
+----+-----------------+
|   5|            28.75|
|   8|17.62857142857143|
|null|23.44017094017094|
|   6|22.82278481012658|
|   4|28.80246913580247|
+----+-----------------+



### Spark API Mini Exercises

**1. Spark Dataframe Basics**

   - Use the starter code above to create a pandas dataframe.
   - Convert the pandas dataframe to a spark dataframe. From this point forward, do all of your work with the spark dataframe, not the pandas dataframe.
   - Show the first 3 rows of the dataframe.
   - Show the first 7 rows of the dataframe.
   - View a summary of the data using .describe.
   - Use .select to create a new dataframe with just the n and abool columns. View the first 5 rows of this dataframe.
   - Use .select to create a new dataframe with just the group and abool columns. View the first 5 rows of this dataframe.
   - Use .select to create a new dataframe with the group column and the abool column renamed to a_boolean_value. Show the first 3 rows of this dataframe.
   - Use .select to create a new dataframe with the group column and the n column renamed to a_numeric_value. Show the first 6 rows of this dataframe.

In [47]:
import pandas as pd
import numpy as np

np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

In [48]:
# Use the starter code above to create a pandas dataframe.
pandas_dataframe

,n,group,abool
0,-0.712391,z,False
1,0.753766,x,False
2,-0.044503,z,False
3,0.451812,y,False
4,1.345102,z,False
5,0.532338,y,False
6,1.350188,z,False
7,0.861211,x,False
8,1.478686,z,True
9,-1.045377,y,True


In [49]:
# Convert the pandas dataframe to a spark dataframe. From this point forward, do all of your work with the spark dataframe, not the pandas dataframe.
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[n: double, group: string, abool: boolean]

In [50]:
# Show the first 3 rows of the dataframe.
df.show(3)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
+--------------------+-----+-----+
only showing top 3 rows



In [51]:
# Show the first 7 rows of the dataframe.
df.show(7)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
+--------------------+-----+-----+
only showing top 7 rows



In [52]:
# View a summary of the data using .describe.
df.describe().show()

+-------+------------------+-----+
|summary|                 n|group|
+-------+------------------+-----+
|  count|                20|   20|
|   mean|0.3664026449885217| null|
| stddev|0.8905322898155363| null|
|    min|-1.261605945319069|    x|
|    max|2.1503829673811126|    z|
+-------+------------------+-----+



In [53]:
# Use .select to create a new dataframe with just the n and abool columns. View the first 5 rows of this dataframe.
df.select('n', 'abool').show(5)

+--------------------+-----+
|                   n|abool|
+--------------------+-----+
|  -0.712390662050588|false|
|   0.753766378659703|false|
|-0.04450307833805...|false|
| 0.45181233874578974|false|
|  1.3451017084510097|false|
+--------------------+-----+
only showing top 5 rows



In [54]:
# Use .select to create a new dataframe with just the group and abool columns. View the first 5 rows of this dataframe.
df.select('group', 'abool').show(5)

+-----+-----+
|group|abool|
+-----+-----+
|    z|false|
|    x|false|
|    z|false|
|    y|false|
|    z|false|
+-----+-----+
only showing top 5 rows



In [55]:
# Use .select to create a new dataframe with the group column and the abool column renamed to a_boolean_value. Show the first 3 rows of this dataframe.
df.select('group', df.abool.alias('a_boolean_value')).show(3)

+-----+---------------+
|group|a_boolean_value|
+-----+---------------+
|    z|          false|
|    x|          false|
|    z|          false|
+-----+---------------+
only showing top 3 rows



In [56]:
# Use .select to create a new dataframe with the group column and the n column renamed to a_numeric_value. Show the first 6 rows of this dataframe.
df.select('group', df.n.alias('a_numeric_value')).show(6)

+-----+--------------------+
|group|     a_numeric_value|
+-----+--------------------+
|    z|  -0.712390662050588|
|    x|   0.753766378659703|
|    z|-0.04450307833805...|
|    y| 0.45181233874578974|
|    z|  1.3451017084510097|
|    y|  0.5323378882945463|
+-----+--------------------+
only showing top 6 rows



**2. Column Manipulation**

   - Use the starter code above to re-create a spark dataframe. Store the spark dataframe in a varaible named df

   - Use .select to add 4 to the n column. Show the results.

   - Subtract 5 from the n column and view the results.

   - Multiply the n column by 2. View the results along with the original numbers.
   - Add a new column named n2 that is the n value multiplied by -1. Show the first 4 rows of your dataframe. You should see the original n value as well as n2.

   - Add a new column named n3 that is the n value squared. Show the first 5 rows of your dataframe. You should see both n, n2, and n3.

   - What happens when you run the code below?
        - df.group + df.abool
    
   - What happens when you run the code below? What is the difference between this and the previous code sample?
        - df.select(df.group + df.abool)
        
   - Try adding various other columns together. What are the results of combining the different data types?

In [57]:
import pandas as pd
import numpy as np

np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

In [58]:
pandas_dataframe

,n,group,abool
0,-0.712391,z,False
1,0.753766,x,False
2,-0.044503,z,False
3,0.451812,y,False
4,1.345102,z,False
5,0.532338,y,False
6,1.350188,z,False
7,0.861211,x,False
8,1.478686,z,True
9,-1.045377,y,True


In [59]:
# Use the starter code above to re-create a spark dataframe. Store the spark dataframe in a varaible named df
data = spark.createDataFrame(pandas_dataframe)
data

DataFrame[n: double, group: string, abool: boolean]

In [60]:
# Use .select to add 4 to the n column. Show the results.
data.select(data.n + 4).show()

+------------------+
|           (n + 4)|
+------------------+
|3.2876093379494122|
| 4.753766378659703|
|3.9554969216619464|
|  4.45181233874579|
|5.3451017084510095|
| 4.532337888294546|
| 5.350187899722527|
|  4.86121137416932|
| 5.478685737435897|
| 2.954622869461466|
|3.2110109750484512|
| 2.738394054680931|
| 4.562846785281032|
|3.7566737481144377|
| 4.913740704859677|
| 4.317350922736336|
| 4.127303280206981|
| 6.150382967381113|
| 4.606288656896298|
|3.9732283500135592|
+------------------+



In [61]:
# Subtract 5 from the n column and view the results.
data.select(data.n - 5).show()

+-------------------+
|            (n - 5)|
+-------------------+
| -5.712390662050588|
| -4.246233621340297|
| -5.044503078338053|
|  -4.54818766125421|
|-3.6548982915489905|
| -4.467662111705454|
|-3.6498121002774733|
|  -4.13878862583068|
| -3.521314262564103|
| -6.045377130538534|
| -5.788989024951549|
| -6.261605945319069|
| -4.437153214718968|
| -5.243326251885563|
| -4.086259295140323|
| -4.682649077263664|
| -4.872696719793019|
|-2.8496170326188874|
| -4.393711343103702|
| -5.026771649986441|
+-------------------+



In [62]:
# Multiply the n column by 2. View the results along with the original numbers.
data.select(data.n, data.n*2).show()

+--------------------+--------------------+
|                   n|             (n * 2)|
+--------------------+--------------------+
|  -0.712390662050588|  -1.424781324101176|
|   0.753766378659703|   1.507532757319406|
|-0.04450307833805...|-0.08900615667610691|
| 0.45181233874578974|  0.9036246774915795|
|  1.3451017084510097|  2.6902034169020195|
|  0.5323378882945463|  1.0646757765890926|
|  1.3501878997225267|  2.7003757994450535|
|  0.8612113741693206|  1.7224227483386412|
|  1.4786857374358966|   2.957371474871793|
| -1.0453771305385342| -2.0907542610770684|
| -0.7889890249515489| -1.5779780499030978|
|  -1.261605945319069|  -2.523211890638138|
|  0.5628467852810314|  1.1256935705620628|
|-0.24332625188556253|-0.48665250377112507|
|  0.9137407048596775|   1.827481409719355|
| 0.31735092273633597|  0.6347018454726719|
| 0.12730328020698067| 0.25460656041396135|
|  2.1503829673811126|   4.300765934762225|
|  0.6062886568962988|  1.2125773137925977|
|-0.02677164998644...|-0.0535432

In [63]:
# Add a new column named n2 that is the n value multiplied by -1. Show the first 4 rows of your dataframe. You should see the original n value as well as n2.
n2 = (data.n * -1).alias('n2')
data.select(data.n, n2).show()

+--------------------+--------------------+
|                   n|                  n2|
+--------------------+--------------------+
|  -0.712390662050588|   0.712390662050588|
|   0.753766378659703|  -0.753766378659703|
|-0.04450307833805...|0.044503078338053455|
| 0.45181233874578974|-0.45181233874578974|
|  1.3451017084510097| -1.3451017084510097|
|  0.5323378882945463| -0.5323378882945463|
|  1.3501878997225267| -1.3501878997225267|
|  0.8612113741693206| -0.8612113741693206|
|  1.4786857374358966| -1.4786857374358966|
| -1.0453771305385342|  1.0453771305385342|
| -0.7889890249515489|  0.7889890249515489|
|  -1.261605945319069|   1.261605945319069|
|  0.5628467852810314| -0.5628467852810314|
|-0.24332625188556253| 0.24332625188556253|
|  0.9137407048596775| -0.9137407048596775|
| 0.31735092273633597|-0.31735092273633597|
| 0.12730328020698067|-0.12730328020698067|
|  2.1503829673811126| -2.1503829673811126|
|  0.6062886568962988| -0.6062886568962988|
|-0.02677164998644...|0.02677164

In [64]:
# Add a new column named n3 that is the n value squared. Show the first 5 rows of your dataframe. You should see both n, n2, and n3.
n3 = (data.n**2).alias('n3')
data.select(data.n, n2, n3).show(5)

+--------------------+--------------------+--------------------+
|                   n|                  n2|                  n3|
+--------------------+--------------------+--------------------+
|  -0.712390662050588|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|0.044503078338053455|0.001980523981562...|
| 0.45181233874578974|-0.45181233874578974| 0.20413438944294027|
|  1.3451017084510097| -1.3451017084510097|  1.8092986060778251|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [65]:
# What happens when you run the code below?
data.group + data.abool
# it shows a column object

Column<b'(group + abool)'>

In [66]:
# What happens when you run the code below? What is the difference between this and the previous code sample?
data.select((data.group + data.abool))
# is will show an error because it is lazy, it hasn't actually performed the aggregation

AnalysisException: "cannot resolve '(CAST(`group` AS DOUBLE) + `abool`)' due to data type mismatch: differing types in '(CAST(`group` AS DOUBLE) + `abool`)' (double and boolean).;;\n'Project [(cast(group#1069 as double) + abool#1070) AS (group + abool)#1120]\n+- LogicalRDD [n#1068, group#1069, abool#1070], false\n"

In [ ]:
# Try adding various other columns together. What are the results of combining the different data types?


### Spark API Mini Exercises

**3. Spark SQL**

   - Use the starter code above to re-create a spark dataframe.
   - Turn your dataframe into a table that can be queried with spark SQL. Name the table my_df. Answer the rest of the questions in this section with a spark sql query (spark.sql) against my_df. After each step, view the first 7 records from the dataframe.
   - Write a query that shows all of the columns from your dataframe.
   - Write a query that shows just the n and abool columns from the dataframe.
   - Write a query that shows just the n and group columns. Rename the group column to g.
   - Write a query that selects n, and creates two new columns: n2, the original n values halved, and n3: the original n values minus 1.
   - What happens if you make a SQL syntax error in your query?

In [98]:
np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

In [99]:
pandas_dataframe.head()

,n,group,abool
0,-0.712391,z,False
1,0.753766,x,False
2,-0.044503,z,False
3,0.451812,y,False
4,1.345102,z,False


In [100]:
my_df = spark.createDataFrame(pandas_dataframe)
my_df.show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
+--------------------+-----+-----+
only showing top 5 rows



In [101]:
# # Turn your dataframe into a table that can be queried with spark SQL. Name the table my_df. Answer the rest of the questions in this section with a spark sql query (spark.sql) against my_df. After each step, view the first 7 records from the dataframe.
my_df.createOrReplaceTempView('my_df')
my_df.show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
|  0.8612113741693206|    x|false|
|  1.4786857374358966|    z| true|
| -1.0453771305385342|    y| true|
| -0.7889890249515489|    x|false|
|  -1.261605945319069|    y|false|
|  0.5628467852810314|    y| true|
|-0.24332625188556253|    y| true|
|  0.9137407048596775|    y|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  2.1503829673811126|    y| true|
|  0.6062886568962988|    x|false|
|-0.02677164998644...|    x| true|
+--------------------+-----+-----+



In [102]:
# Write a query that shows all of the columns from your dataframe.
spark.sql("""
SELECT * FROM my_df
""").show(7)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
+--------------------+-----+-----+
only showing top 7 rows



In [103]:
# Write a query that shows just the n and abool columns from the dataframe.
spark.sql("""
SELECT n, abool FROM my_df
""").show(7)

+--------------------+-----+
|                   n|abool|
+--------------------+-----+
|  -0.712390662050588|false|
|   0.753766378659703|false|
|-0.04450307833805...|false|
| 0.45181233874578974|false|
|  1.3451017084510097|false|
|  0.5323378882945463|false|
|  1.3501878997225267|false|
+--------------------+-----+
only showing top 7 rows



In [104]:
# Write a query that shows just the n and group columns. Rename the group column to g.
spark.sql("""
SELECT n, group AS g FROM my_df
""").show(7)

+--------------------+---+
|                   n|  g|
+--------------------+---+
|  -0.712390662050588|  z|
|   0.753766378659703|  x|
|-0.04450307833805...|  z|
| 0.45181233874578974|  y|
|  1.3451017084510097|  z|
|  0.5323378882945463|  y|
|  1.3501878997225267|  z|
+--------------------+---+
only showing top 7 rows



In [105]:
# Write a query that selects n, and creates two new columns: n2, the original n values halved, and n3: the original n values minus 1.
spark.sql("""
SELECT n, (n / 2) AS n2, (n - 1) AS n3 FROM my_df
""").show(5)

+--------------------+--------------------+--------------------+
|                   n|                  n2|                  n3|
+--------------------+--------------------+--------------------+
|  -0.712390662050588|  -0.356195331025294|  -1.712390662050588|
|   0.753766378659703|  0.3768831893298515|-0.24623362134029703|
|-0.04450307833805...|-0.02225153916902...| -1.0445030783380536|
| 0.45181233874578974| 0.22590616937289487| -0.5481876612542103|
|  1.3451017084510097|  0.6725508542255049| 0.34510170845100974|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [106]:
# What happens if you make a SQL syntax error in your query?
spark.sql("""
SELECT * AS my_df
""")
# the error message shows the error in the syntax

AnalysisException: "Invalid usage of '*' in expression 'alias';"

**4. Type casting**

   - Use the starter code above to re-create a spark dataframe.
   - Use .printSchema to view the datatypes in your dataframe.
   - Use .dtypes to view the datatypes in your dataframe.
   - What is the difference between the two code samples below?
        - df.abool.cast('int')
        - df.select(df.abool.cast('int')).show()
   - Use .select and .cast to convert the abool column to an integer type. View the results.
   - Convert the group column to a integer data type and view the results. What happens?
   - Convert the n column to a integer data type and view the results. What happens?
   - Convert the abool column to a string data type and view the results. What happens?

In [107]:
np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

In [108]:
pandas_dataframe.head(5)

,n,group,abool
0,-0.712391,z,False
1,0.753766,x,False
2,-0.044503,z,False
3,0.451812,y,False
4,1.345102,z,False


In [109]:
df = spark.createDataFrame(pandas_dataframe)
df.show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
+--------------------+-----+-----+
only showing top 5 rows



In [110]:
# Use .printSchema to view the datatypes in your dataframe.
df.printSchema()

root
 |-- n: double (nullable = true)
 |-- group: string (nullable = true)
 |-- abool: boolean (nullable = true)



In [111]:
# Use .dtypes to view the datatypes in your dataframe.
df.dtypes

[('n', 'double'), ('group', 'string'), ('abool', 'boolean')]

In [112]:
# What is the difference between the two code samples below?
df.abool.cast('int')
# This code shows the object changing the dtype

Column<b'CAST(abool AS INT)'>

In [113]:
df.select(df.n, df.n.cast('int')).show(5)

+--------------------+---+
|                   n|  n|
+--------------------+---+
|  -0.712390662050588|  0|
|   0.753766378659703|  0|
|-0.04450307833805...|  0|
| 0.45181233874578974|  0|
|  1.3451017084510097|  1|
+--------------------+---+
only showing top 5 rows



In [114]:
df.select(df.abool.cast('int')).show(5)
# This code shows the column with the dtype changed 

+-----+
|abool|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
+-----+
only showing top 5 rows



In [115]:
# Use .select and .cast to convert the abool column to an integer type. View the results.
df.select(df.abool.cast('int')).show(5)
# the value changed to a numeric value, 1 or 0

+-----+
|abool|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
+-----+
only showing top 5 rows



In [116]:
# Convert the group column to a integer data type and view the results. What happens?
df.select(df.group.cast('int')).show(5)
# group is a string dtype so it becomes null becasue can't convert to int

+-----+
|group|
+-----+
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 5 rows



In [117]:
# Convert the n column to a integer data type and view the results. What happens?
df.select(df.n.cast('int')).show(5)

+---+
|  n|
+---+
|  0|
|  0|
|  0|
|  0|
|  1|
+---+
only showing top 5 rows



In [118]:
# Convert the abool column to a string data type and view the results. What happens?
df.select(df.abool.cast('string')).show(5)

+-----+
|abool|
+-----+
|false|
|false|
|false|
|false|
|false|
+-----+
only showing top 5 rows



**5. Built-in Functions**

   - Use the starter code above to re-create a spark dataframe.
   - Import the necessary functions from pyspark.sql.functions
   - Find the highest n value.
   - Find the lowest n value.
   - Find the average n value.
   - Use concat to change the group column to say, e.g. "Group: x" or "Group: y"
   - Use concat to combine the n and group columns to produce results that look like this: "x: -1.432" or "z: 2.352"

In [120]:
np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

In [121]:
pandas_dataframe.head()

,n,group,abool
0,-0.712391,z,False
1,0.753766,x,False
2,-0.044503,z,False
3,0.451812,y,False
4,1.345102,z,False


In [122]:
df = spark.createDataFrame(pandas_dataframe)
df.show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
+--------------------+-----+-----+
only showing top 5 rows



In [123]:
# Import the necessary functions from pyspark.sql.functions
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean

In [124]:
# Find the highest n value.
df.select(max('n')).show(5)

+------------------+
|            max(n)|
+------------------+
|2.1503829673811126|
+------------------+



In [125]:
# Find the lowest n value.
df.select(min('n')).show()

+------------------+
|            min(n)|
+------------------+
|-1.261605945319069|
+------------------+



In [126]:
# Find the average n value.
df.select(mean('n')).show()
df.select(avg('n')).show()
# both work the same

+------------------+
|            avg(n)|
+------------------+
|0.3664026449885217|
+------------------+

+------------------+
|            avg(n)|
+------------------+
|0.3664026449885217|
+------------------+



In [127]:
from pyspark.sql.functions import lit, round
# lit allows us to use a string literal in the concatenation 

In [128]:
# Use concat to change the group column to say, e.g. "Group: x" or "Group: y"
df.select(concat(lit('Group:'), df.group, lit(" "), df.n)).show(5)

+---------------------------+
|concat(Group:, group,  , n)|
+---------------------------+
|       Group:z -0.712390...|
|       Group:x 0.7537663...|
|       Group:z -0.044503...|
|       Group:y 0.4518123...|
|       Group:z 1.3451017...|
+---------------------------+
only showing top 5 rows



In [134]:
# Use concat to combine the n and group columns to produce results that look like this: "x: -1.432" or "z: 2.352"
df.select(concat(df.group, lit(":"), round(df.n, 3))).show(5)

+-----------------------------+
|concat(group, :, round(n, 3))|
+-----------------------------+
|                     z:-0.712|
|                      x:0.754|
|                     z:-0.045|
|                      y:0.452|
|                      z:1.345|
+-----------------------------+
only showing top 5 rows



In [131]:
df.select(
    concat(lit('Group: '), 'group'),
    concat('group', lit(": "), round(df.n, 3))
    ).show(5)

+----------------------+------------------------------+
|concat(Group: , group)|concat(group, : , round(n, 3))|
+----------------------+------------------------------+
|              Group: z|                     z: -0.712|
|              Group: x|                      x: 0.754|
|              Group: z|                     z: -0.045|
|              Group: y|                      y: 0.452|
|              Group: z|                      z: 1.345|
+----------------------+------------------------------+
only showing top 5 rows



In [135]:
# to bring back the builtin python functions after importing spark functions
__builtins__.min

<function min>

**6. Filter / Where**

   - Use the starter code above to re-create a spark dataframe.
   - Use .filter or .where to select just the rows where the group is y and view the results.
   - Select just the columns where the abool column is false and view the results.
   - Find the columns where the group column is not y.
   - Find the columns where n is positive.
   - Find the columns where abool is true and the group column is z.
   - Find the columns where abool is true or the group column is z.
   - Find the columns where abool is false and n is less than 1
   - Find the columns where abool is false or n is less than 1

In [175]:
np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

In [177]:
pandas_dataframe.head()

,n,group,abool
0,-0.712391,z,False
1,0.753766,x,False
2,-0.044503,z,False
3,0.451812,y,False
4,1.345102,z,False


In [179]:
df = spark.createDataFrame(pandas_dataframe)
df.show(5)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
+--------------------+-----+-----+
only showing top 5 rows



In [184]:
# Use .filter or .where to select just the rows where the group is y and view the results.
df.filter(df.group == 'y').show()
df.where(df.group == 'y').show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
| -1.0453771305385342|    y| true|
|  -1.261605945319069|    y|false|
|  0.5628467852810314|    y| true|
|-0.24332625188556253|    y| true|
|  0.9137407048596775|    y|false|
|  2.1503829673811126|    y| true|
+--------------------+-----+-----+



In [185]:
# Select just the columns where the abool column is false and view the results.
df.filter(df.abool == 'false').show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
|  0.8612113741693206|    x|false|
| -0.7889890249515489|    x|false|
|  -1.261605945319069|    y|false|
|  0.9137407048596775|    y|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
+--------------------+-----+-----+



In [187]:
# Find the columns where the group column is not y.
df.filter(df.group != 'y').show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  0.8612113741693206|    x|false|
|  1.4786857374358966|    z| true|
| -0.7889890249515489|    x|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
|-0.02677164998644...|    x| true|
+--------------------+-----+-----+



In [188]:
# Find the columns where n is positive.
df.filter(df.n > 0).show()

+-------------------+-----+-----+
|                  n|group|abool|
+-------------------+-----+-----+
|  0.753766378659703|    x|false|
|0.45181233874578974|    y|false|
| 1.3451017084510097|    z|false|
| 0.5323378882945463|    y|false|
| 1.3501878997225267|    z|false|
| 0.8612113741693206|    x|false|
| 1.4786857374358966|    z| true|
| 0.5628467852810314|    y| true|
| 0.9137407048596775|    y|false|
|0.31735092273633597|    x|false|
|0.12730328020698067|    z|false|
| 2.1503829673811126|    y| true|
| 0.6062886568962988|    x|false|
+-------------------+-----+-----+



In [190]:
# Find the columns where abool is true and the group column is z.
df.filter((df.abool == 'true') & (df.group == 'z')).show()

+------------------+-----+-----+
|                 n|group|abool|
+------------------+-----+-----+
|1.4786857374358966|    z| true|
+------------------+-----+-----+



In [192]:
# Find the columns where abool is true or the group column is z.
df.filter((df.abool == 'true')|(df.group == 'z')).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|-0.04450307833805...|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  1.4786857374358966|    z| true|
| -1.0453771305385342|    y| true|
|  0.5628467852810314|    y| true|
|-0.24332625188556253|    y| true|
| 0.12730328020698067|    z|false|
|  2.1503829673811126|    y| true|
|-0.02677164998644...|    x| true|
+--------------------+-----+-----+



In [193]:
# Find the columns where abool is false and n is less than 1
df.filter((df.abool == 'false')&(df.n < 1)).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
|  0.8612113741693206|    x|false|
| -0.7889890249515489|    x|false|
|  -1.261605945319069|    y|false|
|  0.9137407048596775|    y|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
+--------------------+-----+-----+



In [194]:
# Find the columns where abool is false or n is less than 1
df.filter((df.abool == 'false')|(df.n < 1)).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
|  0.8612113741693206|    x|false|
| -1.0453771305385342|    y| true|
| -0.7889890249515489|    x|false|
|  -1.261605945319069|    y|false|
|  0.5628467852810314|    y| true|
|-0.24332625188556253|    y| true|
|  0.9137407048596775|    y|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
|-0.02677164998644...|    x| true|
+--------------------+-----+-----+



**7. When / Otherwise**

   - Use the starter code above to re-create a spark dataframe.
   - Use when and .otherwise to create a column that contains the text "It is true" when abool is true and "It is false"" when abool is false.
   - Create a column that contains 0 if n is less than 0, otherwise, the original n value.

In [195]:
np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

In [196]:
df = spark.createDataFrame(pandas_dataframe)
df.show(1)

+------------------+-----+-----+
|                 n|group|abool|
+------------------+-----+-----+
|-0.712390662050588|    z|false|
+------------------+-----+-----+
only showing top 1 row



In [201]:
from pyspark.sql.functions import when

In [206]:
# Use when and .otherwise to create a column that contains the text "It is true" when abool is true and "It is false"" when abool is false.
df.select(
    'abool',
    when(df.abool == 'false', 'It is true')
    .otherwise('It is false').alias('true or false')
).show()

+-----+-------------+
|abool|true or false|
+-----+-------------+
|false|   It is true|
|false|   It is true|
|false|   It is true|
|false|   It is true|
|false|   It is true|
|false|   It is true|
|false|   It is true|
|false|   It is true|
| true|  It is false|
| true|  It is false|
|false|   It is true|
|false|   It is true|
| true|  It is false|
| true|  It is false|
|false|   It is true|
|false|   It is true|
|false|   It is true|
| true|  It is false|
|false|   It is true|
| true|  It is false|
+-----+-------------+



In [208]:
# Create a column that contains 0 if n is less than 0, otherwise, the original n value.
df.select(
    'n',
    when(df.n < 0, 0)
    .otherwise(df.n).alias('original or zero')
).show()

+--------------------+-------------------+
|                   n|   original or zero|
+--------------------+-------------------+
|  -0.712390662050588|                0.0|
|   0.753766378659703|  0.753766378659703|
|-0.04450307833805...|                0.0|
| 0.45181233874578974|0.45181233874578974|
|  1.3451017084510097| 1.3451017084510097|
|  0.5323378882945463| 0.5323378882945463|
|  1.3501878997225267| 1.3501878997225267|
|  0.8612113741693206| 0.8612113741693206|
|  1.4786857374358966| 1.4786857374358966|
| -1.0453771305385342|                0.0|
| -0.7889890249515489|                0.0|
|  -1.261605945319069|                0.0|
|  0.5628467852810314| 0.5628467852810314|
|-0.24332625188556253|                0.0|
|  0.9137407048596775| 0.9137407048596775|
| 0.31735092273633597|0.31735092273633597|
| 0.12730328020698067|0.12730328020698067|
|  2.1503829673811126| 2.1503829673811126|
|  0.6062886568962988| 0.6062886568962988|
|-0.02677164998644...|                0.0|
+----------

**8. Sorting**

   - Use the starter code above to re-create a spark dataframe.
   - Sort by the n value.
   - Sort by the group value, both ascending and descending.
   - Sort by the group value first, then, within each group, sort by n value.
   - Sort by abool, group, and n. Does it matter in what order you specify the columns when sorting?

In [209]:
np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

df = spark.createDataFrame(pandas_dataframe)
df.show(2)

+------------------+-----+-----+
|                 n|group|abool|
+------------------+-----+-----+
|-0.712390662050588|    z|false|
| 0.753766378659703|    x|false|
+------------------+-----+-----+
only showing top 2 rows



In [218]:
from pyspark.sql.functions import asc, desc

In [219]:
# Sort by the n value.
df.sort(df.n).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -1.261605945319069|    y|false|
| -1.0453771305385342|    y| true|
| -0.7889890249515489|    x|false|
|  -0.712390662050588|    z|false|
|-0.24332625188556253|    y| true|
|-0.04450307833805...|    z|false|
|-0.02677164998644...|    x| true|
| 0.12730328020698067|    z|false|
| 0.31735092273633597|    x|false|
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
|  0.5628467852810314|    y| true|
|  0.6062886568962988|    x|false|
|   0.753766378659703|    x|false|
|  0.8612113741693206|    x|false|
|  0.9137407048596775|    y|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  1.4786857374358966|    z| true|
|  2.1503829673811126|    y| true|
+--------------------+-----+-----+



In [220]:
# Sort by the group value, both ascending and descending.
df.sort(df.group.asc()).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  0.6062886568962988|    x|false|
|-0.02677164998644...|    x| true|
|  0.8612113741693206|    x|false|
| 0.31735092273633597|    x|false|
|   0.753766378659703|    x|false|
| -0.7889890249515489|    x|false|
| -1.0453771305385342|    y| true|
|-0.24332625188556253|    y| true|
|  -1.261605945319069|    y|false|
|  0.5323378882945463|    y|false|
| 0.45181233874578974|    y|false|
|  2.1503829673811126|    y| true|
|  0.5628467852810314|    y| true|
|  0.9137407048596775|    y|false|
|-0.04450307833805...|    z|false|
|  1.3501878997225267|    z|false|
|  -0.712390662050588|    z|false|
|  1.3451017084510097|    z|false|
| 0.12730328020698067|    z|false|
|  1.4786857374358966|    z| true|
+--------------------+-----+-----+



In [221]:
df.sort(df.group.desc()).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|  1.3501878997225267|    z|false|
|  1.4786857374358966|    z| true|
|  1.3451017084510097|    z|false|
| 0.12730328020698067|    z|false|
|-0.04450307833805...|    z|false|
|-0.24332625188556253|    y| true|
|  0.9137407048596775|    y|false|
|  -1.261605945319069|    y|false|
|  0.5628467852810314|    y| true|
| -1.0453771305385342|    y| true|
|  0.5323378882945463|    y|false|
|  2.1503829673811126|    y| true|
| 0.45181233874578974|    y|false|
| 0.31735092273633597|    x|false|
|   0.753766378659703|    x|false|
| -0.7889890249515489|    x|false|
|-0.02677164998644...|    x| true|
|  0.8612113741693206|    x|false|
|  0.6062886568962988|    x|false|
+--------------------+-----+-----+



In [225]:
# Sort by the group value first, then, within each group, sort by n value.
df.sort(df.n).sort(df.group).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
| 0.31735092273633597|    x|false|
|  0.6062886568962988|    x|false|
|  0.8612113741693206|    x|false|
| -0.7889890249515489|    x|false|
|-0.02677164998644...|    x| true|
|   0.753766378659703|    x|false|
|  2.1503829673811126|    y| true|
|  0.5323378882945463|    y|false|
| 0.45181233874578974|    y|false|
|-0.24332625188556253|    y| true|
|  0.9137407048596775|    y|false|
|  -1.261605945319069|    y|false|
| -1.0453771305385342|    y| true|
|  0.5628467852810314|    y| true|
| 0.12730328020698067|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  -0.712390662050588|    z|false|
|  1.4786857374358966|    z| true|
|-0.04450307833805...|    z|false|
+--------------------+-----+-----+



In [226]:
# Sort by abool, group, and n. Does it matter in what order you specify the columns when sorting?
df.sort(df.abool).sort(df.group).sort(df.n).show()
# it does not matter the order you group the columns

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -1.261605945319069|    y|false|
| -1.0453771305385342|    y| true|
| -0.7889890249515489|    x|false|
|  -0.712390662050588|    z|false|
|-0.24332625188556253|    y| true|
|-0.04450307833805...|    z|false|
|-0.02677164998644...|    x| true|
| 0.12730328020698067|    z|false|
| 0.31735092273633597|    x|false|
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
|  0.5628467852810314|    y| true|
|  0.6062886568962988|    x|false|
|   0.753766378659703|    x|false|
|  0.8612113741693206|    x|false|
|  0.9137407048596775|    y|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  1.4786857374358966|    z| true|
|  2.1503829673811126|    y| true|
+--------------------+-----+-----+



In [227]:
df.sort(df.n).sort(df.group).sort(df.abool).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  0.9137407048596775|    y|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
| -0.7889890249515489|    x|false|
|  -1.261605945319069|    y|false|
|  1.3501878997225267|    z|false|
|  0.8612113741693206|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  0.5628467852810314|    y| true|
|-0.24332625188556253|    y| true|
|  2.1503829673811126|    y| true|
|-0.02677164998644...|    x| true|
|  1.4786857374358966|    z| true|
| -1.0453771305385342|    y| true|
+--------------------+-----+-----+



**9. Aggregating**

   - What is the average n value for each group in the group column?
   - What is the maximum n value for each group in the group column?
   - What is the minimum n value by abool?
   - What is the average n value for each unique combination of the group and abool column?

In [228]:
df.show(2)

+------------------+-----+-----+
|                 n|group|abool|
+------------------+-----+-----+
|-0.712390662050588|    z|false|
| 0.753766378659703|    x|false|
+------------------+-----+-----+
only showing top 2 rows



In [230]:
# What is the average n value for each group in the group column?
df.groupBy(df.group).agg(avg(df.n)).show()

+-----+------------------+
|group|            avg(n)|
+-----+------------------+
|    x|0.2871427762539448|
|    z| 0.590730814237962|
|    y| 0.257601419602374|
+-----+------------------+



In [231]:
# What is the maximum n value for each group in the group column?
df.groupBy(df.group).agg(max(df.n)).show()

+-----+------------------+
|group|            max(n)|
+-----+------------------+
|    x|0.8612113741693206|
|    z|1.4786857374358966|
|    y|2.1503829673811126|
+-----+------------------+



In [232]:
# What is the minimum n value by abool?
df.groupBy(df.abool).agg(min(df.n)).show()

+-----+-------------------+
|abool|             min(n)|
+-----+-------------------+
| true|-1.0453771305385342|
|false| -1.261605945319069|
+-----+-------------------+



In [ ]:
# What is the average n value for each unique combination of the group and abool column?
df.groupBy(d)

**10. Explain**

**11. Joins**